# Speed

In [ ]:
# default_exp speed

In [ ]:
#export 
import multiprocessing
import threading
import functools
import math
import numpy as np
from numba import njit
import psutil
from multiprocessing import Pool
import os
import logging

import numpy as numpy_
cupy = numpy_
from numba import cuda as cuda_
cuda = cuda_
import numba as numba_
numba = numba_

# We use jit_fun and jit_fun_gpu.
# This is to be able too distinguish GPU device functions and numba optimized functions in a GPU setting

jit_fun = None
jit_fun_gpu = None
speed_mode = None


try:
    import cupy as cupy_
except ModuleNotFoundError:
    cupy_ = None

def dummy_decorator(func):
    """
    Dummy decorator that does nothing
    """
    return func

def set_speed_mode(mode):
    """
    Function to change 
    
    """
    global jit_fun
    global jit_fun_gpu
    global speed_mode
    global cupy
    
    speed_mode = mode
    
    if mode == 'python':
        jit_fun = dummy_decorator
        jit_fun_gpu = dummy_decorator
        cupy = numpy_
    elif mode == 'numba':
        jit_fun = njit
        jit_fun_gpu = njit
        cupy = numpy_
    elif mode == 'numba_gpu':
        jit_fun = njit
        jit_fun_gpu = cuda.jit(device=True)
        if cupy_ is not None:
            cupy = cupy_
        else:
            raise ModuleNotFoundError('Cupy not installed.')
    else:
        raise NotImplementedError(mode)
        
        
if cupy_ is not None:
    set_speed_mode('numba_gpu')
else:
    set_speed_mode('numba')
    
@numba.njit
def grid_1d(x): return -1
@numba.njit
def grid_2d(x): return -1, -1

def set_cuda_grid(dimensions=0):
    global cuda
    if dimensions == 0:
        cuda = cuda_
        cuda.grid = cuda_.grid
    if dimensions == 1:
        cuda = numba_
        cuda.grid = grid_1d
    if dimensions == 2:
        cuda = numba_
        cuda.grid = grid_2d
        
        
def numba_threaded(_func=None, *, cpu_threads=0):
    if cpu_threads <= 0:
        cpu_threads = multiprocessing.cpu_count()
        
    def parallel_compiled_func_inner(func):
        if speed_mode == 'python':
            numba_func = func
        else:
            numba_func = numba.njit(nogil=True)(func)

        def numba_func_parallel(thread, iterable, *args):
            for i in range(thread, len(iterable), cpu_threads):
                numba_func(i, iterable, *args)

        if speed_mode == 'python':
            numba_func_parallel = numba_func_parallel
        else:
            numba_func_parallel = numba.njit(nogil=True)(numba_func_parallel)

        def wrapper(iterable, *args):
            threads = []
            for thread_id in range(cpu_threads):
                t = threading.Thread(
                    target=numba_func_parallel,
                    args=(thread_id, iterable, *args)
                )
                t.start()
                threads.append(t)
            for t in threads:
                t.join()
                del t
        return functools.wraps(func)(wrapper)
    
    if _func is None:
        return parallel_compiled_func_inner
    else:
        return parallel_compiled_func_inner(_func)
    
     
def parallel_compiled_func(
    _func=None,
    *,
    cpu_threads=None,
    dimensions=1,
):
    #set_cuda_grid()
    if dimensions not in (1, 2):
        raise ValueError("Only 1D and 2D are supported")

    if cpu_threads is not None:
        use_gpu = False
    else:
        try:
            cuda_.get_current_device()
        except cuda_.CudaSupportError:
            use_gpu = False
            cpu_threads = 0
        else:
            use_gpu = True
        try:
            import cupy
        except ModuleNotFoundError:
            use_gpu = False
            cpu_threads = 0
            
    if cpu_threads is None:
        cpu_threads = multiprocessing.cpu_count()
        
    if cpu_threads <= 0:
        cpu_threads = multiprocessing.cpu_count()

    if speed_mode == 'numba_gpu':
        use_gpu = True
    elif speed_mode == 'numba':
        use_gpu = False
    elif speed_mode == 'python':
        use_gpu = False
                
    if use_gpu:
        set_cuda_grid()
        def parallel_compiled_func_inner(func):
            cuda_func = cuda.jit(func)
            if dimensions == 1:
                def wrapper(iterable_1d, *args):
                    cuda_func.forall(iterable_1d.shape[0], 1)(
                        -1,
                        iterable_1d,
                        *args
                    )
            elif dimensions == 2:
                def wrapper(iterable_2d, *args):
                    threadsperblock = (
                        min(iterable_2d.shape[0], 16),
                        min(iterable_2d.shape[0], 16)
                    )
                    blockspergrid_x = math.ceil(
                        iterable_2d.shape[0] / threadsperblock[0]
                    )
                    blockspergrid_y = math.ceil(
                        iterable_2d.shape[1] / threadsperblock[1]
                    )
                    blockspergrid = (blockspergrid_x, blockspergrid_y)
                    cuda_func[blockspergrid, threadsperblock](
                        -1,
                        -1,
                        iterable_2d,
                        *args
                    )
            return functools.wraps(func)(wrapper)
    else:
        set_cuda_grid(dimensions)
        if cpu_threads <= 0:
            cpu_threads = multiprocessing.cpu_count()
        def parallel_compiled_func_inner(func):
            
            if speed_mode == 'python':
                numba_func = func
            else:
                numba_func = numba.njit(nogil=True)(func)

            if dimensions == 1:
                def numba_func_parallel(
                    thread,
                    iterable_1d,
                    *args
                ):
                    for i in range(
                        thread,
                        len(iterable_1d),
                        cpu_threads
                    ):
                        numba_func(i, iterable_1d, *args)
            elif dimensions == 2:
                def numba_func_parallel(
                    thread,
                    iterable_2d,
                    *args
                ):
                    for i in range(
                        thread,
                        iterable_2d.shape[0],
                        cpu_threads
                    ):
                        for j in range(iterable_2d.shape[1]):
                            numba_func(i, j, iterable_2d, *args)
                            
            if speed_mode == 'python':
                numba_func_parallel = numba_func_parallel
            else:
                numba_func_parallel = numba.njit(nogil=True)(numba_func_parallel)

            def wrapper(iterable, *args):
                threads = []
                for thread_id in range(cpu_threads):
                    t = threading.Thread(
                        target=numba_func_parallel,
                        args=(thread_id, iterable, *args)
                    )
                    t.start()
                    threads.append(t)
                for t in threads:
                    t.join()
                    del t
            return functools.wraps(func)(wrapper)
    if _func is None:
        return parallel_compiled_func_inner
    else:
        return parallel_compiled_func_inner(_func)
    
def set_max_process(a):
    max_processes = psutil.cpu_count()
    new_max = min(a, max_processes)
    
    return new_max

def AlphaPool(a, *args, **kwargs):  
    max_processes = psutil.cpu_count()
    if a == -1:
        a = max_processes
    new_max = min(a, 50, max_processes)
    print(f"AlphaPool was set to {a} processes. Setting max to {new_max}.")

    return Pool(new_max)

In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_settings.ipynb.
Converted 01_chem.ipynb.
Converted 02_io.ipynb.
Converted 03_fasta.ipynb.
Converted 04_feature_finding.ipynb.
Converted 04_feature_finding2.ipynb.
Converted 05_search.ipynb.
Converted 06_score.ipynb.
Converted 07_recalibration.ipynb.
Converted 08_quantification.ipynb.
Converted 09_matching.ipynb.
Converted 10_constants.ipynb.
Converted 11_interface.ipynb.
Converted 12_speed.ipynb.
Converted index.ipynb.
Converted minimal_example.ipynb.
Converted minimal_example_2.ipynb.
Converted minimal_example_3.ipynb.
Converted test.ipynb.
Converted Untitled.ipynb.
Converted Untitled1.ipynb.
Converted Untitled2.ipynb.
